In [1]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

In [27]:
nielsen15 = pd.read_csv('../../Nielsen/aggregated_nielsen_2015.csv')
nielsen16 = pd.read_csv('../..//Nielsen/aggregated_nielsen_2016.csv')
nielsen15['year'] = 2015
nielsen16['year'] = 2016

In [28]:
nielsen = pd.concat((nielsen15, nielsen16))
nielsen = nielsen[~nielsen.is_walmart]

In [29]:
## Entriy/exit dates
fandom = pd.read_csv('../data_collection/plein_de_data/fandom_traitées.csv', parse_dates=['Opening_date', 'Closing_date'])[['State', 'County_name', 'County_fips', 'Opening_date', 'Closing_date']]

# We drop the state in which we do not trust our data (some mistakes stillremain)
fandom = fandom[~np.isin(fandom.State, ('CA', 'GA', 'KS', 'LA', 'TX'))]
nielsen = nielsen[~np.isin(nielsen.store_state, ('CA', 'GA', 'KS', 'LA', 'TX'))]

# We concentrate our study on the movements (entries & exits) during the fiscal years 2015 and 2016
movements = fandom[((fandom.Opening_date >= '2015-01-31') & (fandom.Opening_date <= '2017-01-31')) | ((fandom.Closing_date >= '2015-01-31') & (fandom.Closing_date <= '2017-01-31'))]
#movements['year'] = movements.Opening_date.dt.year
#movements['month'] = movements.Opening_date.dt.month

In [74]:
nielsen.groupby('product_group_descr').mean().sort_values('upc_price_std', ascending=True).head(10)

,is_walmart,guessed_store_county_fips,purchase_year,purchase_month,upc_price,upc_price_std,nb_of_obs,year
product_group_descr,,,,,,,,
YEAST,0.0,34358.913876,2015.550239,6.937799,3.457727,0.190346,1.186603,2015.550239
"JUICES, DRINKS-FROZEN",0.0,31276.076638,2015.491695,6.359750,1.825015,0.440367,4.133884,2015.491695
ICE,0.0,29888.888000,2015.499846,6.668000,2.318100,0.465500,2.631077,2015.499846
VEGETABLES - CANNED,0.0,31244.049491,2015.498625,6.517163,1.038096,0.517872,29.851215,2015.498625
PASTA,0.0,31698.057646,2015.500946,6.462988,1.449016,0.546721,15.659901,2015.500946
"PUDDING, DESSERTS-DAIRY",0.0,31518.914590,2015.495125,6.446009,2.434284,0.547729,4.569520,2015.495125
FRUIT - CANNED,0.0,31497.732971,2015.496603,6.641208,1.654077,0.633034,10.802289,2015.496603
DOUGH PRODUCTS,0.0,31231.945841,2015.496236,6.548513,1.972625,0.634339,8.207640,2015.496236
SOUP,0.0,31434.031000,2015.497921,6.540429,1.506098,0.698717,25.787703,2015.497921


In [76]:
categories = [
    "FRESH PRODUCE",
    "BREAD AND BAKED GOODS",
    "MILK",
    "SNACKS",
    "PACKAGED MEATS-DELI",
    "CHEESE",
    "UNPREP MEAT/POULTRY/SEAFOOD-FRZN",
    "CARBONATED BEVERAGES",
    "CONDIMENTS, GRAVIES, AND SAUCES",
    "CANDY",
    "JUICE, DRINKS - CANNED, BOTTLED",
    "EGGS",
    "CEREAL",
    "PASTA",
]

## PREMIER MODELE

$$Prices_{i, t} = \alpha + \beta treat_i + \gamma treat_i* post_t$$

In [77]:
# We choose to focus on milk prices
for category in categories:
    product_group = nielsen[nielsen.product_group_descr == category]


    # The control group is composed by all states where nothing (no entry nor exit) happened.
    control = product_group[~np.isin(product_group.guessed_store_county_fips, movements)].copy()
    print(f"Size of the control group: {len(control.guessed_store_county_fips.unique())}.")


    # The treatment group is composed by the states where one entry took place in 2016 and where this entry is the only movement
    count = movements.groupby('County_fips').count()
    count = count[count.State == 1] # No more than one movement in the treatement group
    treatment_movements = movements[(np.isin(movements.County_fips, count.index))]
    treatment_movements = treatment_movements[(treatment_movements.Opening_date>='2015-01-31' ) & (treatment_movements.Opening_date<='2017-01-31') & ((treatment_movements.Closing_date>'2017-01-31') | (treatment_movements.Closing_date.apply(str) == 'NaT'))]

    treatment = product_group[np.isin(product_group.guessed_store_county_fips, treatment_movements.County_fips )].copy()
    treatment = treatment.merge(treatment_movements, left_on='guessed_store_county_fips', right_on='County_fips')
    print(f"Size of the treatment group: {len(treatment.guessed_store_county_fips.unique())}.")


    # We create our dummies for the regression
    control['treat'] = False
    control['interaction'] = False

    treatment['treat'] = True
    treatment['interaction'] = (treatment.purchase_month > treatment.Opening_date.dt.month) & (treatment.purchase_year >= treatment.Opening_date.dt.year)


    # Final dataset for the regression :

    df = pd.concat((control, treatment))[['upc_price', 'treat', 'interaction']]
    df = df[df.upc_price != 0]
    reg0 = smf.ols(formula='np.log(upc_price) ~ treat + interaction', data=df)
    results0 = reg0.fit()
    if abs(results0.params[2] / results0.bse[2]) > 1. :
        print("=========================================================")
        print(category)
        print(f"Coef : {np.exp(results0.params[2])-1}")
        print(f"Coef/err : {abs(results0.params[2] / results0.bse[2])}")
        print(f"CI_up : {np.exp(results0.conf_int(alpha=0.05)[0][2])-1}")
        print(f"CI_down : {np.exp(results0.conf_int(alpha=0.05)[1][2])-1}")

Size of the control group: 1831.
Size of the treatment group: 91.
FRESH PRODUCE
Coef : -0.019621247262988195
Coef/err : 1.54306197746831
CI_up : -0.04399037238436465
CI_down : 0.005369058066230448
Size of the control group: 1836.
Size of the treatment group: 91.
Size of the control group: 1822.
Size of the treatment group: 91.
MILK
Coef : -0.04808014547348827
Coef/err : 4.722368379334597
CI_up : -0.0673505450061016
CI_down : -0.028411581018182752
Size of the control group: 1837.
Size of the treatment group: 91.
Size of the control group: 1816.
Size of the treatment group: 91.
Size of the control group: 1816.
Size of the treatment group: 91.
CHEESE
Coef : -0.027670775680054494
Coef/err : 2.1548667078264496
CI_up : -0.05217409316341537
CI_down : -0.0025339952755393247
Size of the control group: 1794.
Size of the treatment group: 90.
Size of the control group: 1815.
Size of the treatment group: 91.
CARBONATED BEVERAGES
Coef : -0.01938904684432885
Coef/err : 1.0296924514499477
CI_up : -0.0